In [13]:
import os

from datetime import datetime

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, DateTime, Integer
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine(os.environ["DB_CONN_STR"])
Session = sessionmaker(bind=engine)

Base = declarative_base()

class Calls2(Base):
    __tablename__ = 'calls2'
    ts = Column(DateTime,primary_key=True)
    msisdn = Column(String,primary_key=True)
    val = Column(Integer)
    
    
Base.metadata.create_all(engine)

In [14]:
session = Session()

sql = '''
SELECT create_hypertable('calls2', 'ts', 'msisdn', chunk_time_interval => interval '1 year',migrate_data => true)
'''

session.execute(sql)
session.commit()

In [15]:
from faker import Faker
from random import randint
fake = Faker()

In [17]:
session = Session()
for i in range(10):
    rows = [Calls2(ts = fake.date_object(end_datetime=None), msisdn = f"{fake.catch_phrase()}_{str(fake.date_object(end_datetime=None))}",val=randint(0,1000)) for _ in range(10000)]
    session.bulk_save_objects(rows)
    session.commit()

In [23]:
from sqlalchemy import func
session = Session()
year_bucket = func.time_bucket('365 days', Calls.ts)
count = func.count(Calls.msisdn).desc()
session.query(year_bucket, func.count(Calls.msisdn)).group_by(year_bucket).order_by(count).all()

[(datetime.datetime(1973, 1, 9, 0, 0), 2168),
 (datetime.datetime(1975, 1, 9, 0, 0), 2132),
 (datetime.datetime(2004, 1, 2, 0, 0), 2100),
 (datetime.datetime(1983, 1, 7, 0, 0), 2092),
 (datetime.datetime(1986, 1, 6, 0, 0), 2085),
 (datetime.datetime(1982, 1, 7, 0, 0), 2081),
 (datetime.datetime(1993, 1, 4, 0, 0), 2078),
 (datetime.datetime(2003, 1, 2, 0, 0), 2075),
 (datetime.datetime(1992, 1, 5, 0, 0), 2072),
 (datetime.datetime(1970, 1, 10, 0, 0), 2071),
 (datetime.datetime(1994, 1, 4, 0, 0), 2070),
 (datetime.datetime(1998, 1, 3, 0, 0), 2066),
 (datetime.datetime(1977, 1, 8, 0, 0), 2063),
 (datetime.datetime(1981, 1, 7, 0, 0), 2059),
 (datetime.datetime(2011, 12, 31, 0, 0), 2055),
 (datetime.datetime(1991, 1, 5, 0, 0), 2052),
 (datetime.datetime(1988, 1, 6, 0, 0), 2051),
 (datetime.datetime(2007, 1, 1, 0, 0), 2050),
 (datetime.datetime(2014, 12, 30, 0, 0), 2048),
 (datetime.datetime(1979, 1, 8, 0, 0), 2045),
 (datetime.datetime(1999, 1, 3, 0, 0), 2045),
 (datetime.datetime(1996, 1, 

In [24]:
year = func.date_trunc('year', Calls2.ts)
hist = func.histogram(Calls2.val,0, 1000, 10)
session.query(year,hist).group_by(year).all()

[(datetime.datetime(2001, 1, 1, 0, 0),
  [0, 191, 210, 203, 213, 240, 205, 216, 215, 211, 185, 1]),
 (datetime.datetime(1972, 1, 1, 0, 0),
  [0, 203, 203, 200, 220, 198, 213, 192, 204, 201, 220, 2]),
 (datetime.datetime(1991, 1, 1, 0, 0),
  [0, 217, 208, 200, 207, 194, 198, 194, 186, 188, 196, 0]),
 (datetime.datetime(1990, 1, 1, 0, 0),
  [0, 210, 186, 221, 211, 203, 229, 203, 193, 222, 190, 3]),
 (datetime.datetime(2012, 1, 1, 0, 0),
  [0, 210, 224, 209, 195, 192, 190, 220, 189, 192, 214, 1]),
 (datetime.datetime(2008, 1, 1, 0, 0),
  [0, 209, 224, 204, 220, 194, 217, 191, 198, 221, 197, 0]),
 (datetime.datetime(2002, 1, 1, 0, 0),
  [0, 206, 202, 188, 189, 207, 197, 215, 225, 214, 199, 3]),
 (datetime.datetime(1977, 1, 1, 0, 0),
  [0, 196, 179, 173, 207, 179, 202, 213, 218, 202, 207, 4]),
 (datetime.datetime(2015, 1, 1, 0, 0),
  [0, 206, 218, 205, 214, 201, 217, 197, 191, 219, 204, 1]),
 (datetime.datetime(2009, 1, 1, 0, 0),
  [0, 210, 201, 198, 194, 205, 188, 210, 202, 208, 219, 2]),


In [27]:
first_val = func.first(Calls2.val, Calls2.ts)
year = func.date_trunc('year', Calls2.ts)
session.query(year, first_val).group_by(year).all()

[(datetime.datetime(2001, 1, 1, 0, 0), 695),
 (datetime.datetime(1972, 1, 1, 0, 0), 484),
 (datetime.datetime(1991, 1, 1, 0, 0), 561),
 (datetime.datetime(1990, 1, 1, 0, 0), 18),
 (datetime.datetime(2012, 1, 1, 0, 0), 308),
 (datetime.datetime(2008, 1, 1, 0, 0), 459),
 (datetime.datetime(2002, 1, 1, 0, 0), 752),
 (datetime.datetime(1977, 1, 1, 0, 0), 877),
 (datetime.datetime(2015, 1, 1, 0, 0), 491),
 (datetime.datetime(2009, 1, 1, 0, 0), 250),
 (datetime.datetime(2003, 1, 1, 0, 0), 165),
 (datetime.datetime(2018, 1, 1, 0, 0), 713),
 (datetime.datetime(1973, 1, 1, 0, 0), 773),
 (datetime.datetime(1996, 1, 1, 0, 0), 976),
 (datetime.datetime(2010, 1, 1, 0, 0), 124),
 (datetime.datetime(1992, 1, 1, 0, 0), 487),
 (datetime.datetime(1979, 1, 1, 0, 0), 794),
 (datetime.datetime(1976, 1, 1, 0, 0), 718),
 (datetime.datetime(2005, 1, 1, 0, 0), 412),
 (datetime.datetime(1997, 1, 1, 0, 0), 416),
 (datetime.datetime(1971, 1, 1, 0, 0), 327),
 (datetime.datetime(2013, 1, 1, 0, 0), 211),
 (datetime.

In [29]:
x = func.chunk_relation_size_pretty('calls2')
session.query(x).all()

[('(51,_timescaledb_internal._hyper_2_51_chunk,{ts},"{""timestamp without time zone""}",{NULL},"{""[\'1972-12-16 00:00:00+00\',\'1973-12-11 00:00:00+00\')""}","200 kB","320 kB","8192 bytes","528 kB")'),
 ('(52,_timescaledb_internal._hyper_2_52_chunk,{ts},"{""timestamp without time zone""}",{NULL},"{""[\'1994-08-23 00:00:00+00\',\'1995-08-18 00:00:00+00\')""}","208 kB","320 kB","8192 bytes","536 kB")'),
 ('(53,_timescaledb_internal._hyper_2_53_chunk,{ts},"{""timestamp without time zone""}",{NULL},"{""[\'2017-04-24 00:00:00+00\',\'2018-04-19 00:00:00+00\')""}","200 kB","288 kB","8192 bytes","496 kB")'),
 ('(54,_timescaledb_internal._hyper_2_54_chunk,{ts},"{""timestamp without time zone""}",{NULL},"{""[\'2009-06-05 00:00:00+00\',\'2010-05-31 00:00:00+00\')""}","200 kB","288 kB","8192 bytes","496 kB")'),
 ('(55,_timescaledb_internal._hyper_2_55_chunk,{ts},"{""timestamp without time zone""}",{NULL},"{""[\'1980-11-04 00:00:00+00\',\'1981-10-30 00:00:00+00\')""}","200 kB","304 kB","8192 bytes